In [203]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook
import sympy as sp
from IPython.display import display
from sympy import Eq, Symbol

In [204]:


# Set the reference geometry to be generated from the closed form kinematics
t_num = 15
a_num= 1000/4
b_num= 1615/4
c_num= 645/4
e_num= 286/4
gamma_num= 0.683


wb_shell = WBTessellation4PEx(
                         a = a_num,
                         b = b_num, 
                         c = c_num, 
                         e_x = e_num,
                         gamma=gamma_num, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                         n_phi_plus=2, # planned 5 
                         n_x_plus=2,  # planned 3
                         wireframe_width=5,
                        ##---- Trimming function works only in WBTessellation4P ----##
                         trim_half_cells_along_y=True,
                         trim_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
# wb_shell.interact()
orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)

In [205]:
orig_I_Fi

array([[ 0,  2,  1],
       [ 1,  2,  3],
       [ 0,  1,  4],
       [ 1,  5,  4],
       [ 0,  4,  6],
       [ 0,  6,  2],
       [ 1,  7,  5],
       [ 1,  3,  7],
       [ 8,  9,  7],
       [ 9, 10,  7],
       [ 8,  7,  3],
       [11,  7, 12],
       [12,  7, 10],
       [11,  5,  7],
       [13, 14, 15],
       [14,  6, 15],
       [14,  2,  6],
       [16, 15, 17],
       [17, 15,  6],
       [17,  6,  4]])

Derivation of the crease nodes of the lower facet (-)

![Crease convention and nomenclature](wb_nomenclature_analytical.png)

In [206]:
# Define symbols
a, b, c, e, gamma, t = sp.symbols(r'a b c e \gamma t', real=True)

alpha = sp.pi/2 - gamma


In [207]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
}

subs_dict

{a: 250.0, b: 403.75, c: 161.25, e: 71.5, \gamma: 0.683, t: 15}

In [208]:
# Known angles (alpha_x and alpha_z)
theta_x = sp.Symbol('theta_x', real=True)
theta_y = sp.Symbol('theta_y', real=True)  # Unknown
theta_z = sp.Symbol('theta_z', real=True)

# Trig functions
cx = sp.cos(theta_x)
sx = sp.sin(theta_x)
cy = sp.cos(theta_y)
sy = sp.sin(theta_y)
cz = sp.cos(theta_z)
sz = sp.sin(theta_z)

# Rotation matrices
R_x = sp.Matrix([
    [1, 0, 0],
    [0, cx, -sx],
    [0, sx, cx]
])
R_y = sp.Matrix([
    [cy, 0, sy],
    [0, 1, 0],
    [-sy, 0, cy]
])
R_z = sp.Matrix([
    [cz, -sz, 0],
    [sz, cz, 0],
    [0, 0, 1]
])

# Full rotation matrix (Z * Y * X)
#R = R_z @ R_y @ R_x
R_x


Matrix([
[1,            0,             0],
[0, cos(theta_x), -sin(theta_x)],
[0, sin(theta_x),  cos(theta_x)]])

In [209]:
x_folded = a+e
display(Eq(sp.Symbol(r'x^{\urcorner}'), x_folded))

Eq(x^{\urcorner}, a + e)

In [210]:
y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
display(Eq(sp.Symbol(r'y^{\urcorner}'), y_folded))

Eq(y^{\urcorner}, sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))

In [211]:
z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)
display(Eq(sp.Symbol(r'z^{\urcorner}'), z_folded))

Eq(z^{\urcorner}, a*(1 - sin(\gamma))/cos(\gamma))

In [212]:
# Calculating theta to follow the curvature of the tesselation
theta = ((z_folded - c*sp.cos(gamma))**2- y_folded**2) / ((z_folded - c*sp.cos(gamma))**2+ y_folded**2)
theta = -sp.pi +sp.acos(theta.simplify())
display(Eq(Symbol(r'\theta_x'), theta, evaluate=False)) 

Eq(\theta_x, acos((-a**2*(sin(\gamma) - 1)**2 + b**2*cos(\gamma)**2 - (a*(sin(\gamma) - 1) + c*cos(\gamma)**2)**2)/((-2*a*c*sin(\gamma) + 2*a*c - b**2 - c**2*cos(\gamma)**2)*cos(\gamma)**2)) - pi)

In [213]:
theta_x_num = np.array(theta.subs(subs_dict)).astype(np.float64).flatten().tolist()[0]
theta_x_num

-0.03208748520889472

In [214]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
    theta_x: theta_x_num,
}

subs_dict

{a: 250.0,
 b: 403.75,
 c: 161.25,
 e: 71.5,
 \gamma: 0.683,
 t: 15,
 theta_x: -0.03208748520889472}

In [215]:
O_gamma_center_lower = sp.Matrix([0, 0, 0])
display(Eq(Symbol(r'O_{\gamma-}^{+}'), O_gamma_center_lower, evaluate=False))

Eq(O_{\gamma-}^{+}, Matrix([
[0],
[0],
[0]]))

In [216]:
C_gamma_mr_lower = sp.Matrix([e, 0, 0])
C_gamma_mr_lower
display(Eq(Symbol(r'C_{\gamma-}^{\dashv}'), C_gamma_mr_lower, evaluate=False))

Eq(C_{\gamma-}^{\dashv}, Matrix([
[e],
[0],
[0]]))

In [217]:
C_gamma_mr_lower_num = np.array(C_gamma_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_mr_lower_num, orig_X_Ia[0]

([71.5, 0.0, 0.0], array([71.5,  0. ,  0. ]))

In [218]:
C_gamma_ml_lower = sp.Matrix([-e, 0, 0])
C_gamma_ml_lower
display(Eq(Symbol(r'C_{\gamma-}^{\vdash}'), C_gamma_ml_lower, evaluate=False))

Eq(C_{\gamma-}^{\vdash}, Matrix([
[-e],
[ 0],
[ 0]]))

In [219]:
C_gamma_ml_lower_num = np.array(C_gamma_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_ml_lower_num, orig_X_Ia[1]

([-71.5, 0.0, 0.0], array([-71.5,   0. ,   0. ]))

In [220]:
V_gamma_mr_lower = sp.Matrix([e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'V_{\gamma-}^{\dashv}'), V_gamma_mr_lower, evaluate=False))

Eq(V_{\gamma-}^{\dashv}, Matrix([
[c*sin(\gamma) + e],
[                0],
[    c*cos(\gamma)]]))

In [221]:
V_gamma_mr_lower_num = np.array(V_gamma_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_mr_lower_num, orig_X_Ia[6]

([173.26856909385202, 0.0, 125.07885850370505],
 array([173.26856995,   0.        , 125.07910156]))

In [222]:
V_gamma_ml_lower = sp.Matrix([-e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'V_{\gamma-}^{\dashv}'), V_gamma_ml_lower, evaluate=False))

Eq(V_{\gamma-}^{\dashv}, Matrix([
[-c*sin(\gamma) - e],
[                 0],
[     c*cos(\gamma)]]))

In [223]:
V_gamma_ml_lower_num = np.array(V_gamma_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_ml_lower_num, orig_X_Ia[7]

([-173.26856909385202, 0.0, 125.07885850370505],
 array([-173.26856995,    0.        ,  125.07910156]))

In [224]:
U_gamma_ur_lower = sp.Matrix([x_folded, y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\urcorner}'), U_gamma_ur_lower, evaluate=False))

Eq(U_{\gamma-}^{\urcorner}, Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]]))

In [225]:
U_gamma_ur_lower_num = np.array(U_gamma_ur_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_ur_lower_num, orig_X_Ia[2]

([321.5, 385.8493740048242, 118.88785926277468],
 array([321.5       , 385.84936523, 118.88867188]))

In [226]:
U_gamma_ul_lower = sp.Matrix([-x_folded, y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\ulcorner}'), U_gamma_ur_lower, evaluate=False))

Eq(U_{\gamma-}^{\ulcorner}, Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]]))

In [227]:
U_gamma_ul_lower_num = np.array(U_gamma_ul_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_ul_lower_num, orig_X_Ia[3]

([-321.5, 385.8493740048242, 118.88785926277468],
 array([-321.5       ,  385.84936523,  118.88867188]))

In [228]:
U_gamma_ll_lower = sp.Matrix([-x_folded, -y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\llcorner}'), U_gamma_ll_lower, evaluate=False))

Eq(U_{\gamma-}^{\llcorner}, Matrix([
[                                                             -a - e],
[-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [229]:
U_gamma_ll_lower_num = np.array(U_gamma_ll_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_ll_lower_num, orig_X_Ia[5]

([-321.5, -385.8493740048242, 118.88785926277468],
 array([-321.5       , -385.84936523,  118.88867188]))

In [230]:
U_gamma_lr_lower = sp.Matrix([x_folded, -y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\llcorner}'), U_gamma_ll_lower, evaluate=False))

Eq(U_{\gamma-}^{\llcorner}, Matrix([
[                                                             -a - e],
[-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [231]:
U_gamma_lr_lower_num = np.array(U_gamma_lr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_lr_lower_num, orig_X_Ia[4]

([321.5, -385.8493740048242, 118.88785926277468],
 array([ 321.5       , -385.84936523,  118.88867188]))

In [232]:
delta_theta_ur = V_gamma_mr_lower - R_x * U_gamma_ll_lower
display(Eq(Symbol(r'\delta_{\theta}^{\urcorner}'), delta_theta_ur, evaluate=False))

Eq(\delta_{\theta}^{\urcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [233]:
delta_theta_ul = V_gamma_ml_lower - R_x * U_gamma_lr_lower
display(Eq(Symbol(r'\delta_{\theta}^{\ulcorner}'), delta_theta_ul, evaluate=False))

Eq(\delta_{\theta}^{\ulcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [234]:
M_gamma_ur_lower = R_x @ -C_gamma_mr_lower + delta_theta_ur
display(Eq(Symbol(r'M_{\gamma-}^{\urcorner}'), M_gamma_ur_lower, evaluate=False))

Eq(M_{\gamma-}^{\urcorner}, Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [235]:
M_gamma_ur_lower_num = np.array(M_gamma_ur_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
M_gamma_ur_lower_num, orig_X_Ia[14]

([423.268569093852, 381.8365966624425, -6.126613749715986],
 array([423.26856909, 381.83660889,  -6.12597656]))

In [236]:
M_gamma_ul_lower = M_gamma_ur_lower.multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'M_{\gamma-}^{\ulcorner}'), M_gamma_ul_lower, evaluate=False))

Eq(M_{\gamma-}^{\ulcorner}, Matrix([
[                                                                                                                         -a - c*sin(\gamma) - e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [237]:
M_gamma_ul_lower_num = np.array(M_gamma_ul_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
M_gamma_ul_lower_num, orig_X_Ia[8]

([-423.268569093852, 381.8365966624425, -6.126613749715986],
 array([-423.26856909,  381.83660889,   -6.12597656]))

In [238]:
M_gamma_ll_lower = M_gamma_ur_lower.multiply_elementwise(sp.Matrix([-1, -1, 1]))
display(Eq(Symbol(r'M_{\gamma-}^{\llcorner}'), M_gamma_ll_lower, evaluate=False))

Eq(M_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                         -a - c*sin(\gamma) - e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [239]:
M_gamma_ll_lower_num = np.array(M_gamma_ll_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
M_gamma_ll_lower_num, orig_X_Ia[11]

([-423.268569093852, -381.8365966624425, -6.126613749715986],
 array([-423.26856909, -381.83660889,   -6.12597656]))

In [240]:
M_gamma_lr_lower = M_gamma_ur_lower.multiply_elementwise(sp.Matrix([1, -1, 1]))
display(Eq(Symbol(r'M_{\gamma-}^{\lrcorner}'), M_gamma_lr_lower, evaluate=False))

Eq(M_{\gamma-}^{\lrcorner}, Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [241]:
M_gamma_lr_lower_num = np.array(M_gamma_lr_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
M_gamma_lr_lower_num, orig_X_Ia[17]

([423.268569093852, -381.8365966624425, -6.126613749715986],
 array([ 423.26856909, -381.83660889,   -6.12597656]))

In [242]:
H_gamma_ur_lower = delta_theta_ur
display(Eq(Symbol(r'H_{\gamma-}^{\urcorner}'), H_gamma_ur_lower, evaluate=False))

Eq(H_{\gamma-}^{\urcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [243]:
H_gamma_ur_lower_num = np.array(H_gamma_ur_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_gamma_ur_lower_num, orig_X_Ia[13]

([494.768569093852, 381.8365966624425, -6.126613749715986],
 array([494.76856909, 381.83660889,  -6.12597656]))

In [244]:
H_gamma_ul_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\ulcorner}'), H_gamma_ul_lower, evaluate=False))

Eq(H_{\gamma-}^{\ulcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [245]:
H_gamma_ul_lower_num = np.array(H_gamma_ul_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_gamma_ul_lower_num, orig_X_Ia[9]

([-494.768569093852, 381.8365966624425, -6.126613749715986],
 array([-494.76856909,  381.83660889,   -6.12597656]))

In [246]:
H_gamma_ll_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, -1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\llcorner}'), H_gamma_ll_lower, evaluate=False))

Eq(H_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [247]:
H_gamma_ll_lower_num = np.array(H_gamma_ll_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_gamma_ll_lower_num, orig_X_Ia[12]

([-494.768569093852, -381.8365966624425, -6.126613749715986],
 array([-494.76856909, -381.83660889,   -6.12597656]))

In [248]:
H_gamma_lr_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([1, -1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\llcorner}'), H_gamma_lr_lower, evaluate=False))

Eq(H_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [249]:
H_gamma_lr_lower_num = np.array(H_gamma_lr_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_gamma_lr_lower_num, orig_X_Ia[16]

([494.768569093852, -381.8365966624425, -6.126613749715986],
 array([ 494.76856909, -381.83660889,   -6.12597656]))

In [250]:
H_gamma_mr_lower = sp.Matrix([a + 2*e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'H_{\gamma-}^{\dashv}'), H_gamma_mr_lower, evaluate=False))

Eq(H_{\gamma-}^{\dashv}, Matrix([
[a + c*sin(\gamma) + 2*e],
[                      0],
[          c*cos(\gamma)]]))

In [251]:
H_gamma_mr_lower_num = np.array(H_gamma_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
H_gamma_mr_lower_num, orig_X_Ia[15]

([494.768569093852, 0.0, 125.07885850370505],
 array([ 4.94768554e+02, -7.62939453e-06,  1.25079102e+02]))

In [252]:
H_gamma_ml_lower = sp.Matrix([-a-2*e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'H_{\gamma-}^{\vdash}'), H_gamma_ml_lower, evaluate=False))

Eq(H_{\gamma-}^{\vdash}, Matrix([
[-a - c*sin(\gamma) - 2*e],
[                       0],
[           c*cos(\gamma)]]))

In [253]:
H_gamma_ml_lower_num = np.array(H_gamma_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
H_gamma_ml_lower_num, orig_X_Ia[10]

([-494.768569093852, 0.0, 125.07885850370505],
 array([-4.94768554e+02, -7.62939453e-06,  1.25079102e+02]))

In [254]:
O_gamma_center_upper = sp.Matrix([0, 0, t])
display(Eq(Symbol(r'O_{\gamma+}^{+}'), O_gamma_center_upper, evaluate=False))

Eq(O_{\gamma+}^{+}, Matrix([
[0],
[0],
[t]]))

In [255]:
O_gamma_ml_lower_num = np.array(O_gamma_center_upper.subs(subs_dict)).astype(np.float64).flatten()
O_gamma_ml_lower_num

array([ 0.,  0., 15.])

In [256]:
C_gamma_mr_upper = sp.Matrix([e-t*sp.tan(alpha/2), 0, t])
display(Eq(Symbol(r'C_{\gamma+}^{\dashv}'), C_gamma_mr_upper, evaluate=False))


Eq(C_{\gamma+}^{\dashv}, Matrix([
[e - t*cot(\gamma/2 + pi/4)],
[                         0],
[                         t]]))

In [257]:
C_gamma_mr_upper_num = np.array(C_gamma_mr_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_mr_upper_num

[64.36672844423352, 0.0, 15.0]

In [258]:
V_gamma_mr_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), 0, c*sp.cos(gamma)+t])
display(Eq(Symbol(r'V_{\gamma+}^{\dashv}'), V_gamma_mr_upper, evaluate=False))


Eq(V_{\gamma+}^{\dashv}, Matrix([
[c*sin(\gamma) + e - t*cot(\gamma/2 + pi/4)],
[                                         0],
[                         c*cos(\gamma) + t]]))

In [259]:
V_gamma_mr_upper_num = np.array(V_gamma_mr_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_mr_upper_num

[166.13529753808555, 0.0, 140.07885850370505]

In [260]:
H_gamma_mr_upper = sp.Matrix([a+2*e+c*sp.sin(gamma),0,  c*sp.cos(gamma)+t])
display(Eq(Symbol(r'H_{\gamma+}^{\dashv}'), H_gamma_mr_upper, evaluate=False))


Eq(H_{\gamma+}^{\dashv}, Matrix([
[a + c*sin(\gamma) + 2*e],
[                      0],
[      c*cos(\gamma) + t]]))

In [261]:
C_gamma_ml_upper = sp.Matrix([e-t*sp.tan(alpha/2), 0, t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'C_{\gamma+}^{\vdash}'), C_gamma_ml_upper, evaluate=False))

Eq(C_{\gamma+}^{\vdash}, Matrix([
[-e + t*cot(\gamma/2 + pi/4)],
[                          0],
[                          t]]))

In [262]:
V_gamma_ml_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), 0, c*sp.cos(gamma)+t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'V_{\gamma+}^{\vdash}'), V_gamma_ml_upper, evaluate=False))

Eq(V_{\gamma+}^{\vdash}, Matrix([
[-c*sin(\gamma) - e + t*cot(\gamma/2 + pi/4)],
[                                          0],
[                          c*cos(\gamma) + t]]))

In [263]:
H_gamma_ml_upper = sp.Matrix([a+2*e+c*sp.sin(gamma),0,  c*sp.cos(gamma)+t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'H_{\gamma+}^{\vdash}'), H_gamma_ml_upper, evaluate=False))


Eq(H_{\gamma+}^{\vdash}, Matrix([
[-a - c*sin(\gamma) - 2*e],
[                       0],
[       c*cos(\gamma) + t]]))

In [264]:
import k3d
import numpy as np

def plot_lines_with_k3d(points, connectivity):
    """
    Plot lines with k3d given points and connectivity.

    Parameters:
    - points: (N, 3) array-like of 3D coordinates
    - connectivity: (M, 2) array-like of index pairs defining lines
    """
    points = np.array(points, dtype=np.float32)
    connectivity = np.array(connectivity, dtype=np.uint32)

    # Flatten the points into a line vertex sequence
    lines = k3d.lines(
        vertices=points,
        indices=connectivity,
        shader='simple',
        width=0.01
    )

    plot = k3d.plot()
    plot += lines
    plot.display()

# Example usage:
points = [
    C_gamma_mr_lower_num, # 0
    C_gamma_ml_lower_num, # 1
    U_gamma_ur_lower_num, # 2
    U_gamma_ul_lower_num, # 3
    U_gamma_lr_lower_num, # 4
    U_gamma_ll_lower_num, # 5
    V_gamma_mr_lower_num, # 6
    V_gamma_ml_lower_num, # 7
    M_gamma_ul_lower_num, # 8
    H_gamma_ul_lower_num, # 9
    H_gamma_ml_lower_num, # 10
    M_gamma_ll_lower_num, # 11
    H_gamma_ll_lower_num, # 12
    H_gamma_ur_lower_num, # 13
    M_gamma_ur_lower_num, # 14
    H_gamma_mr_lower_num, # 15
    H_gamma_lr_lower_num, # 16
    M_gamma_lr_lower_num, # 17
    ]

# upper facets


connectivity = [
# lower facets
[ 0,  2,  1],
[ 1,  2,  3],
[ 0,  1,  4],
[ 1,  5,  4],
[ 0,  4,  6],
[ 0,  6,  2],
[ 1,  7,  5],
[ 1,  3,  7],
[ 8,  9,  7],
[ 9, 10,  7],
[ 8,  7,  3],
[11,  7, 12],
[12,  7, 10],
[11,  5,  7],
[13, 14, 15],
[14,  6, 15],
[14,  2,  6],
[16, 15, 17],
[17, 15,  6],
[17,  6,  4],

#upper facets

]



def plot_mesh_with_k3d(points, triangles):
    points = np.array(points, dtype=np.float32)
    triangles = np.array(triangles, dtype=np.uint32)

    # Duplicate the triangles with reversed vertex order
    triangles_reversed = triangles[:, [0, 2, 1]]
    all_triangles = np.vstack([triangles, triangles_reversed])

    mesh = k3d.mesh(
        vertices=points,
        indices=all_triangles,
        wireframe=False,
        color=0x1E90FF,
        opacity=1.0
    )

    plot = k3d.plot()
    plot += mesh
    plot.display()
# Call the plotting function
plot_mesh_with_k3d(points, connectivity)

Output()

In [265]:
y = H_gamma_mr_lower[1] - H_gamma_ur_lower[1]
z = H_gamma_mr_lower[2] - H_gamma_ur_lower[2]

In [266]:
cos_beta = y / sp.sqrt(y**2 + z**2)
cos_beta

(-a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))/sqrt((-a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2 + (a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2)

In [267]:
h = t / cos_beta
h

t*sqrt((-a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2 + (a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2)/(-a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))

In [268]:
h_num = np.array(h.subs(subs_dict))
h_num

array(-15.8608421846844, dtype=object)

In [269]:
# Given C_gamma_mr_lower_num, U_gamma_ur_lower_num, U_gamma_ul_lower_num, calculate the unit normal vector
def calculate_normal_vector(p1, p2, p3):
    # Convert points to numpy arrays
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)

    # Calculate two vectors in the plane
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate the cross product to get the normal vector
    normal_vector = np.cross(v1, v2)

    # Normalize the normal vector
    normal_vector /= np.linalg.norm(normal_vector)

    return normal_vector
# Example points
p1 = np.array(C_gamma_mr_lower_num)
p2 = np.array(U_gamma_ul_lower_num)
p3 = np.array(U_gamma_ur_lower_num)
# Calculate the normal vector
normal_vector = calculate_normal_vector(p1, p2, p3)
normal_vector

array([ 0.        ,  0.29445909, -0.95566408])

In [270]:
l = 0.7
b = 0.015
f = 30

sig = f / (l * b)
sig

2857.1428571428573

In [ ]:
reinf_unit_length = 1000